# Защита персональных данных клиентов

Нужно защитить данные клиентов страховой компании «Хоть потоп». Необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. А также обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. 

## Загрузка данных

In [69]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [70]:
df = pd.read_csv('/datasets/insurance.csv')

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [72]:
df.columns=['sex', 'age', 'salary', 'family', 'payments']
df

,sex,age,salary,family,payments
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
- Целевой признак: количество страховых выплат клиенту за последние 5 лет.

In [73]:
df['age'].describe()

count    5000.000000
mean       30.952800
std         8.440807
min        18.000000
25%        24.000000
50%        30.000000
75%        37.000000
max        65.000000
Name: age, dtype: float64

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** 'исходные' предсказания, полученные по формуле $ a = Xw $, равны 'новым' предсказаниям, полученным по формуле  $ a_1 = XPw_1 $, где P - произвольная обратимая, квадратная матрица.

**Обоснование:** ...

$$
(A B C)^{-1} = (B C)^{-1} A^{-1} = C^{-1} B^{-1} A^{-1}
$$
$$
a=Xw
$$
$$
a_1=XPw_1
$$


$$
X (X^T X)^{-1} X^T y = X P ((XP)^T XP)^{-1} (XP)^T y 
$$
$$
X (X^T X)^{-1} X^T y = X P (P^T X^T XP)^{-1} ((XP)^T) y
$$
$$
X (X^T X)^{-1} X^T y = X P (P^T (X^T X)P)^{-1} P^T  X^T y 
$$
$$
X (X^T X)^{-1} X^T y = X P P^{-1} (X^T X)^{-1} P^{-T} P^T X^T y
$$
$$
A A^{-1} = A^{-1} A = E 
$$
$$
A E = E A = A
$$
$$
X (X^T X)^{-1} X^T y = X (X^T X)^{-1} X^T y
$$

**Обоснование**
Как следует из доказательства формулы на шаге 2, алгоритм шифрования - это умножение матрицы признаков на матрицу P, произвольную обратимую квадратную матрицу.

## Проверка алгоритма

- Cоздаем рандомную матрицу, проверяем ее на обратимость, создаем 2 модели. 
- Первую обучаем на матрице признаков и предсказываем на ней же. 
- Вторую обучаем на зашифрованной матрице признаков (X* P), и на ней же предсказываем. 
- Затем считаем для обеих метрики R2 и сравниваем их, должны совпасть.

In [74]:
def matrix(m,n):
    matrix = np.random.normal(size=(m,n))
    while np.linalg.det(matrix) == 0:
        if m==n:
            matrix = np.random.normal(size=(m,n))
    return matrix

In [75]:
P = matrix(4,4)
P

array([[-0.73502885,  0.91183628,  0.77716335,  0.85585965],
       [-0.01563265, -1.5098163 , -0.52419843,  1.69406677],
       [ 0.43737871,  0.42699444, -0.98634645, -0.99099499],
       [ 0.96828394,  0.056703  , -0.29017208,  0.87425151]])

In [76]:
f = df.drop(columns=['payments']).values 
t = df['payments'].values
model1 = LinearRegression()
model1.fit(f,t)
model2 = LinearRegression()
model2.fit(f @ P, t)
test1 = model1.predict(df.loc[10:20].drop(columns=['payments']).values)
test2 = model2.predict(df.loc[10:20].drop(columns=['payments']).values @ P)
display(r2_score(test1, df.loc[10:20]['payments'].values))
display(r2_score(test2, df.loc[10:20]['payments'].values))

-0.3854150326866359

-0.3854150326775203

## Вывод: <br>
при преобразовании качество моделей машинного обучения не ухудшилось.

тут есть два влияющих на результат момента: то, что sklearn считает регрессию при помощи приближения, а не перемножения матриц поэтому модели сошлись немного по разному, ну и операции с float тоже накапливают небольшую погрешность. Если посчитать регрессию на матрицах, разница будет совсем мизерной. 